# Project 3: Do sellers with more product variety generate higher monthly revenue

- **Dataset(s) to be used:** Brazilian E-Commerce Public Dataset by Olist[[link](https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce)]
- **Analysis question:** [Do sellers with more product variety (larger catalogs) generate higher monthly revenue?]
- using multiple datasets **Columns that will (likely) be used:**
  - from[order_items]: [seller_id], , [product_id], [price], [shipping_limit_date]
  - from[products]: [product_id], [product_category_name]
  - from[orders]: [order_id], [order_purchase_timestamp]
- **Columns to be used to merge/join them:**
  - [products] [order_items]
  - [orders] [order_items]
- **Hypothesis**: [Sellers with larger product catalogs generate higher total monthly revenue.]

## Data loading and sample definition

The Olist dataset provides detailed micro-level information on Brazilian e-commerce orders, including items, sellers, products and order status.  
For this project, I focus on three main tables:

- `orders`: overall information for each order (status, timestamps)
- `order_items`: individual items within orders (seller, product, price)
- `products`: product-level information (category, dimensions)

I restrict the sample to orders with status `"delivered"` in order to measure *realized* revenue rather than including cancelled or unavailable orders.  
I then convert the purchase timestamp into a monthly period (`purchase_month`) so that I can aggregate revenue at the seller–month level.

In [ ]:
import pandas as pd

orders = pd.read_csv("olist_orders_dataset.csv")
order_items = pd.read_csv("olist_order_items_dataset.csv")
products = pd.read_csv("olist_products_dataset.csv")

In [7]:
orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [3]:
order_items.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [4]:
products.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


### Constructing seller-level metrics

To test the hypothesis, I need two seller-level indicators:

1. **Product variety (catalog size)**  
   For each seller, I count the number of distinct `product_id` values that appear in the `order_items` table.  
   This gives me `num_unique_products`, which measures how many different products a seller actually sells through the platform.

In [9]:
seller_variety = (
    order_items.groupby("seller_id")["product_id"]
    .nunique()
    .reset_index(name="num_unique_products")
)

seller_variety

,seller_id,num_unique_products
0,0015a82c2db000af6aaaf3ae2ecb0532,1
1,001cca7ae9ae17fb1caed9dfb1094831,11
2,001e6ad469a905060d959994f1b41e4f,1
3,002100f778ceb8431b7a1020ff7ab48f,24
4,003554e2dce176b5555353e4f3555ac8,1
...,...,...
3090,ffcfefa19b08742c5d315f2791395ee5,1
3091,ffdd9f82b9a447f6f8d4b91554cc7dd3,12
3092,ffeee66ac5d5a62fe688b9d26f83f534,3
3093,fffd5413c0700ac820c7069d66d98c89,29


2. **Average monthly revenue**  
   I first merge `order_items` with the filtered `orders` table to attach a `purchase_month` to each item.  
   Then I compute the total revenue per seller and month by summing the `price` column.  
   Finally, I average these monthly revenues across all active months for each seller, obtaining `avg_monthly_revenue`.

In [19]:
orders_delivered = orders[orders["order_status"] == "delivered"].copy()

orders_delivered["order_purchase_timestamp"] = pd.to_datetime(orders["order_purchase_timestamp"])
orders_delivered["purchase_month"] = orders["order_purchase_timestamp"].dt.to_period("M").astype(str)
orders_delivered.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,purchase_month
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,2017-10
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,2018-07
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,2018-08
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,2017-11
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,2018-02


In [21]:
order_items_merged = order_items.merge(
    orders_delivered[["order_id", "purchase_month"]],
    on="order_id",
    how="inner"   # 用 inner 更干净，只保留有对应订单的行
)
order_items_merged

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,purchase_month
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,2017-09
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,2017-04
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,2018-01
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,2018-08
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,2017-02
...,...,...,...,...,...,...,...,...
110192,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41,2018-04
110193,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53,2018-07
110194,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95,2017-10
110195,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72,2017-08


In [12]:
monthly_revenue = (
    order_items_merged.groupby(["seller_id", "purchase_month"])["price"]
    .sum()
    .reset_index(name="monthly_revenue")
)
monthly_revenue

,seller_id,purchase_month,monthly_revenue
0,0015a82c2db000af6aaaf3ae2ecb0532,2017-09,895.00
1,0015a82c2db000af6aaaf3ae2ecb0532,2017-10,1790.00
2,001cca7ae9ae17fb1caed9dfb1094831,2017-02,1098.90
3,001cca7ae9ae17fb1caed9dfb1094831,2017-03,1676.70
4,001cca7ae9ae17fb1caed9dfb1094831,2017-04,1847.20
...,...,...,...
16436,fffd5413c0700ac820c7069d66d98c89,2018-08,485.40
16437,ffff564a4f9085cd26170f4732393726,2016-10,120.50
16438,ffff564a4f9085cd26170f4732393726,2017-01,238.50
16439,ffff564a4f9085cd26170f4732393726,2017-03,963.35


In [13]:
seller_revenue = (
    monthly_revenue.groupby("seller_id")["monthly_revenue"]
    .mean()
    .reset_index(name="avg_monthly_revenue")
)
seller_revenue

,seller_id,avg_monthly_revenue
0,0015a82c2db000af6aaaf3ae2ecb0532,1342.500000
1,001cca7ae9ae17fb1caed9dfb1094831,1475.295882
2,001e6ad469a905060d959994f1b41e4f,250.000000
3,002100f778ceb8431b7a1020ff7ab48f,154.312500
4,003554e2dce176b5555353e4f3555ac8,120.000000
...,...,...
3090,ffcfefa19b08742c5d315f2791395ee5,69.900000
3091,ffdd9f82b9a447f6f8d4b91554cc7dd3,191.018182
3092,ffeee66ac5d5a62fe688b9d26f83f534,262.837143
3093,fffd5413c0700ac820c7069d66d98c89,647.307143


These two indicators are then merged into a single DataFrame called `seller_stats`, which contains one row per seller with:

- `num_unique_products`
- `avg_monthly_revenue`

This DataFrame is the basis for the visualization and regression analysis.

In [25]:
seller_stats = seller_variety.merge(
    seller_revenue,
    on="seller_id",
    how="inner"
)

seller_stats.describe()

,num_unique_products,avg_monthly_revenue
count,3095.000000,3095.000000
mean,11.130210,555.533105
std,24.473252,1104.721263
min,1.000000,3.500000
25%,2.000000,102.485417
50%,4.000000,242.200000
75%,10.000000,534.754444
max,399.000000,18564.670833


In [24]:
seller_stats[["num_unique_products", "avg_monthly_revenue"]].corr()

,num_unique_products,avg_monthly_revenue
num_unique_products,1.000000,0.434837
avg_monthly_revenue,0.434837,1.000000


The correlation matrix shows a positive correlation between catalog size and average monthly revenue.This suggests that sellers offering more unique products tend to generate higher average monthly revenue.

In [29]:
import plotly.express as px
from IPython.display import HTML

fig_hist_variety = px.histogram(
    seller_stats,
    x="num_unique_products",
    nbins=50,
    title="Distribution of Seller Catalog Size",
    labels={"num_unique_products": "Number of Unique Products"}
)
fig_hist_variety.show()

Most sellers have relatively small catalogs, while a few sellers offer a very large number of products.

In [33]:
import numpy as np

bins = [0, 5, 10, 20, 50, 100, seller_stats["num_unique_products"].max()]
labels = ["1–5", "6–10", "11–20", "21–50", "51–100", "100+"]

seller_stats["variety_bin"] = pd.cut(
    seller_stats["num_unique_products"],
    bins=bins,
    labels=labels,
    include_lowest=True
)

fig_box = px.box(
    seller_stats,
    x="variety_bin",
    y="avg_monthly_revenue",
    title="Average Monthly Revenue by Catalog Size Group",
    labels={
        "variety_bin": "Catalog Size Group (Number of Unique Products)",
        "avg_monthly_revenue": "Average Monthly Revenue"
    }
)

HTML(fig_box.to_html(include_plotlyjs="cdn", full_html=False))

The boxplot shows that median monthly revenue generally increases with catalog size.
Sellers in the highest catalog-size groups tend to have both higher median revenue and a wider upper tail.

## Visualization: product variety vs monthly revenue
I use a scatter plot to visualize the relationship between product variety and monthly revenue.  
Each point represents one seller, with:

- x-axis: `num_unique_products` (catalog size)
- y-axis: `avg_monthly_revenue`

To summarize the overall pattern, I add an OLS trendline using Plotly Express.  
The scatter plot allows me to see the distribution of sellers, while the trendline shows whether the average relationship between catalog size and revenue is positive or negative.

In [32]:
import plotly.express as px
from IPython.display import HTML

fig = px.scatter(
    seller_stats,
    x="num_unique_products",
    y="avg_monthly_revenue",
    opacity=0.5,
    trendline="ols",
    labels={
        "num_unique_products": "Unique Products (Catalog Size)",
        "avg_monthly_revenue": "Average Monthly Revenue"
    },
    title="Product Variety vs Monthly Revenue for Sellers"
)

HTML(fig.to_html(include_plotlyjs="cdn", full_html=False))

Visually, the cloud of points seems to slope upward, suggesting a positive association between catalog size and monthly revenue.

## Regression results

In [17]:
results = px.get_trendline_results(fig)

# Extract the statsmodels OLS results from the DataFrame
model = results.iloc[0]["px_fit_results"]

# Display regression summary
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.189
Model:                            OLS   Adj. R-squared:                  0.189
Method:                 Least Squares   F-statistic:                     721.2
Date:                Fri, 05 Dec 2025   Prob (F-statistic):          5.62e-143
Time:                        15:01:33   Log-Likelihood:                -25755.
No. Observations:                3095   AIC:                         5.151e+04
Df Residuals:                    3093   BIC:                         5.153e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        337.0638     19.648     17.155      0.000     298.539     375.588
x1            19.6285      0.731     26.855      0.000      18.195      21.062
==============================================================================
Omnibus:                     3747.861   Durbin-Watson:                   2.055
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           631820.984
Skew:                           6.312   Prob(JB):                         0.00
Kurtosis:                      71.848   Cond. No.                         29.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The regression results strongly support the hypothesis that sellers with larger product catalogs tend to generate higher monthly revenue. The estimated slope coefficient is 19.63, meaning that each additional unique product a seller offers is associated with an increase of approximately 19.6 BRL in average monthly revenue. This effect is statistically significant at the 1% level (p < 0.001), as shown by the very large t-statistic (26.86).

Although the R-squared value is 0.189, the positive and highly significant coefficient suggests a clear upward relationship between catalog size and revenue. In other words, while other factors also influence seller performance (such as product quality, logistics, ratings, or pricing), product variety remains an important and meaningful predictor on its own.

The fitted regression line therefore indicates that expanding the number of products offered on the platform is, on average, associated with higher monthly sales for sellers. 